In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, root_mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
import nltk
from nltk.corpus import stopwords
from datetime import datetime
import unicodedata

## Agregar o quantitativo de terceirizados por contrato

Como o foco é o quantitativo total por contrato, já que o percentual será aplicado nas vagas totais, vamos agregar o quantitativo de terceirizados por contrato.

In [2]:
df_contratos = pd.read_csv('../data/contratos/categoria_contrato_v2.csv', low_memory=False, dtype='str')

In [3]:
df_contratos.head()

,nr_contrato,nm_categoria_profissional,vl_mensal_salario_mean,vl_mensal_salario_median,vl_mensal_custo_mean,vl_mensal_custo_median,nr_jornada_mean,nr_jornada_median,id_terc_count,sg_orgao_sup_tabela_ug_x,...,id_contrato,objeto_contrato,objeto_compra,dataAssinatura,dataPublicacaoDOU,dataInicioVigencia,dataFimVigencia,tipo_fornecedor,valorInicialCompra,valorFinalCompra
0,000012012,517330 - VIGILANTE,1775.2800000000002,1775.28,3923.92,3923.92,40.0,40.0,46,MINIST.,...,668336396.0,Objeto: Prestação de serviço de vigilancia d...,Objeto: Pregão Eletrônico - Contratação de em...,2012-06-18,2012-07-05,2012-06-18,2013-06-18,Entidades Empresariais Privadas,687542.0,2660693.36
1,000012014,513315 - CAMAREIRO DE HOTEL,1549.11,1549.11,4281.87,4281.87,44.0,44.0,4,MP.,...,668318167.0,Objeto: Contratação de empresa especializada n...,Objeto: Pregão Eletrônico - Contratação de em...,2009-01-12,2009-01-13,2009-01-12,2010-01-11,Entidades Empresariais Privadas,913810.56,5687893.2
2,000012014,514315 - LIMPADOR DE FACHADAS,1549.11,1549.11,4281.87,4281.87,44.0,44.0,1,MP.,...,668318167.0,Objeto: Contratação de empresa especializada n...,Objeto: Pregão Eletrônico - Contratação de em...,2009-01-12,2009-01-13,2009-01-12,2010-01-11,Entidades Empresariais Privadas,913810.56,5687893.2
3,000012014,622015 - TRABALHADOR NA PRODUCAO DE MUDAS E SE...,1549.11,1549.11,4281.87,4281.87,44.0,44.0,1,MP.,...,668318167.0,Objeto: Contratação de empresa especializada n...,Objeto: Pregão Eletrônico - Contratação de em...,2009-01-12,2009-01-13,2009-01-12,2010-01-11,Entidades Empresariais Privadas,913810.56,5687893.2
4,000012016,313120 - TECNICO DE MANUTENCAO ELETRICA,4131.04,4131.04,9030.21,9030.21,30.0,30.0,5,MCTI,...,668296909.0,Objeto: Prestação de serviços de Limpeza e Con...,"Objeto: Pregão Eletrônico - Contratação,em re...",2011-01-03,2011-02-11,2011-01-03,2012-01-03,Entidades Empresariais Privadas,764634.12,4775950.54


In [4]:
df_contratos.shape

(918, 41)

In [5]:
df_contratos.columns

Index(['nr_contrato', 'nm_categoria_profissional', 'vl_mensal_salario_mean',
       'vl_mensal_salario_median', 'vl_mensal_custo_mean',
       'vl_mensal_custo_median', 'nr_jornada_mean', 'nr_jornada_median',
       'id_terc_count', 'sg_orgao_sup_tabela_ug_x', 'cd_ug_gestora_x',
       'nm_ug_tabela_ug_x', 'sg_ug_gestora_x', 'nm_razao_social',
       'nm_unidade_prestacao_x', 'sg_orgao_x', 'nm_orgao_x', 'cd_orgao_siafi',
       'cd_orgao_siape_x', 'cnpj_formatado_x', 'sg_orgao_sup_tabela_ug_y',
       'cd_ug_gestora_y', 'nm_ug_tabela_ug_y', 'sg_ug_gestora_y', 'nr_cnpj',
       'nm_unidade_prestacao_y', 'sg_orgao_y', 'nm_orgao_y',
       'cd_orgao_siape_y', 'cnpj_formatado_y', 'status_validacao',
       'id_contrato', 'objeto_contrato', 'objeto_compra', 'dataAssinatura',
       'dataPublicacaoDOU', 'dataInicioVigencia', 'dataFimVigencia',
       'tipo_fornecedor', 'valorInicialCompra', 'valorFinalCompra'],
      dtype='object')

In [6]:
df_contratos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 41 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   nr_contrato                918 non-null    object
 1   nm_categoria_profissional  918 non-null    object
 2   vl_mensal_salario_mean     918 non-null    object
 3   vl_mensal_salario_median   918 non-null    object
 4   vl_mensal_custo_mean       918 non-null    object
 5   vl_mensal_custo_median     918 non-null    object
 6   nr_jornada_mean            918 non-null    object
 7   nr_jornada_median          918 non-null    object
 8   id_terc_count              918 non-null    object
 9   sg_orgao_sup_tabela_ug_x   918 non-null    object
 10  cd_ug_gestora_x            918 non-null    object
 11  nm_ug_tabela_ug_x          918 non-null    object
 12  sg_ug_gestora_x            912 non-null    object
 13  nm_razao_social            918 non-null    object
 14  nm_unidade

In [7]:
# Remove espaços e converte diretamente para int, usando erros='coerce' para NaNs
df_contratos['id_terc_count'] = (
    df_contratos['id_terc_count']
    .astype(str)  # garante string
    .str.replace(r'\D', '', regex=True)  # remove tudo que não for dígito
    .replace('', np.nan)  # evita strings vazias após regex
    .astype(float)  # ainda float, para aceitar NaN
    .astype('Int64')  # pandas nullable integer
)

In [8]:
df_contratos['id_terc_count'].nlargest(10)

595    2312
241    1961
442    1920
357    1843
284    1240
391    1144
574    1000
153     952
804     881
267     840
Name: id_terc_count, dtype: Int64

In [9]:
df_contrato_agg = df_contratos.groupby(['nr_contrato', 'cd_orgao_siafi']).agg({
    'id_terc_count': 'sum',
    'objeto_contrato': 'first',
    'valorInicialCompra': 'first',
    'valorFinalCompra': 'first',
    'dataAssinatura': 'first',
    'dataInicioVigencia': 'first',
    'dataFimVigencia': 'first'
}).reset_index()

In [10]:
df_contrato_agg.head(10)

,nr_contrato,cd_orgao_siafi,id_terc_count,objeto_contrato,valorInicialCompra,valorFinalCompra,dataAssinatura,dataInicioVigencia,dataFimVigencia
0,000012012,42207,46,Objeto: Prestação de serviço de vigilancia d...,687542.0,2660693.36,2012-06-18,2012-06-18,2013-06-18
1,000012014,20202,6,Objeto: Contratação de empresa especializada n...,913810.56,5687893.2,2009-01-12,2009-01-12,2010-01-11
2,000012016,20301,15,Objeto: Prestação de serviços de Limpeza e Con...,764634.12,4775950.54,2011-01-03,2011-01-03,2012-01-03
3,000012016,26438,6,Objeto: Servicos especializados de Recepcao pa...,118666.32,129217.44,2010-11-03,2010-11-22,2011-11-21
4,000012017,20301,68,Objeto: Prestação de serviços de Limpeza e Con...,764634.12,4775950.54,2011-01-03,2011-01-03,2012-01-03
5,000012017,26416,28,Objeto: Contratação de empresa para prestação ...,280709.88,280709.88,2013-12-06,2013-12-06,2014-12-06
6,000012017,26437,102,Objeto: Contratacao de empresa especializada n...,640741.94,4015923.0,2010-07-01,2010-07-01,2011-07-01
7,000012018,26201,2,Objeto: Limpeza e higienização e conservação d...,7900.0,7900.0,2013-09-09,2013-09-09,2013-10-23
8,000012018,26405,62,Objeto: Contratação de empresa especializada p...,110054.88,110054.88,2012-08-06,2012-08-06,2013-08-05
9,000012019,26406,7,Objeto: Contratação dos serviços de recepcioni...,69408.0,313801.94,2011-05-11,2011-05-11,2015-08-26


In [11]:
df_contrato_agg.shape

(368, 9)

## Tratamento dos dados para os modelos

#### Tratar valores nulos e com string

In [12]:
df_contrato_agg.drop(columns=['nr_contrato'], inplace=True)

In [13]:
df_contrato_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   cd_orgao_siafi      368 non-null    object
 1   id_terc_count       368 non-null    Int64 
 2   objeto_contrato     368 non-null    object
 3   valorInicialCompra  368 non-null    object
 4   valorFinalCompra    368 non-null    object
 5   dataAssinatura      368 non-null    object
 6   dataInicioVigencia  368 non-null    object
 7   dataFimVigencia     368 non-null    object
dtypes: Int64(1), object(7)
memory usage: 23.5+ KB


In [14]:
# Conversão de colunas numéricas
for col in ['valorInicialCompra', 'valorFinalCompra']:
    df_contrato_agg[col] = pd.to_numeric(df_contrato_agg[col], errors='coerce')
    
# Conversão de datas
for col in ['dataAssinatura', 'dataInicioVigencia', 'dataFimVigencia']:
    df_contrato_agg[col] = pd.to_datetime(df_contrato_agg[col], errors='coerce')

In [15]:
df_contrato_agg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 368 entries, 0 to 367
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   cd_orgao_siafi      368 non-null    object        
 1   id_terc_count       368 non-null    Int64         
 2   objeto_contrato     368 non-null    object        
 3   valorInicialCompra  368 non-null    float64       
 4   valorFinalCompra    368 non-null    float64       
 5   dataAssinatura      368 non-null    datetime64[ns]
 6   dataInicioVigencia  368 non-null    datetime64[ns]
 7   dataFimVigencia     368 non-null    datetime64[ns]
dtypes: Int64(1), datetime64[ns](3), float64(2), object(2)
memory usage: 23.5+ KB


#### Filtrar pelos contratos com mais de 25 contratações

Segundo o § 1º do Art. 3º do Decreto nº 11.430/2023, o percentual de 8% será aplicável apenas a contratos com mais de 25 colaboradores.

In [16]:
# Filtra apenas contratos com mais de 25 terceirizados
df_contrato_agg = df_contrato_agg[df_contrato_agg['id_terc_count'] > 25]

In [17]:
df_contrato_agg.shape

(154, 8)

#### Criação de novas variáveis

In [18]:
# Feature de data: ano, mês e intervalo
df_contrato_agg['ano_assinatura'] = df_contrato_agg['dataAssinatura'].dt.year
df_contrato_agg['mes_assinatura'] = df_contrato_agg['dataAssinatura'].dt.month

In [19]:
# Diferença entre assinatura e início da vigência
df_contrato_agg['dias_ate_inicio_vigencia'] = (df_contrato_agg['dataInicioVigencia'] - df_contrato_agg['dataAssinatura']).dt.days

In [20]:
df_contrato_agg.columns

Index(['cd_orgao_siafi', 'id_terc_count', 'objeto_contrato',
       'valorInicialCompra', 'valorFinalCompra', 'dataAssinatura',
       'dataInicioVigencia', 'dataFimVigencia', 'ano_assinatura',
       'mes_assinatura', 'dias_ate_inicio_vigencia'],
      dtype='object')

#### Tratar as stopwords para não influenciarem o modelo

In [21]:
# Preencher nulos e limpar texto do campo de objeto
df_contrato_agg['objeto_contrato'] = df_contrato_agg['objeto_contrato'].fillna("")
df_contrato_agg['objeto_contrato'] = (
    df_contrato_agg['objeto_contrato']
    .str.replace(r"(?i)^objeto:\s*", "", regex=True)  # Remove "Objeto:"
    .str.replace(r"[^a-zA-Záéíóúâêîôûãõç\s]", " ", regex=True)  # Remove pontuação e números
    .str.lower()
    .str.strip()
)

In [22]:
# Preparar stopwords customizadas
def normalize(text):
    return unicodedata.normalize("NFKD", text).encode("ascii", "ignore").decode("utf-8").lower()

nltk.download('stopwords')
stopwords_custom = [normalize(w) for w in stopwords.words('portuguese')]
stopwords_custom.extend([normalize(p) for p in [
    "objeto", "serviços", "prestação", "empresa", "serviço",
    "especializada", "contratação", "edital", "fornecimento", "campus",
    "conforme", "deste", "termo", "anexo", "referencia", "ifsc", "especializados",
    "referente", "especificacoes", "porto", "ifrs", "terceirizados", "alegre",
    "item", "florianopolis", "ibiruba", "erechim", "sertao", "restinga",
    "pregao", "seguir", "aositens", "tabela", "atividades", "mg", "federal",
    "contrato", "areas", "12x36", "serem", "incluindo", "executados", "pessoa",
    "especificacao", "joinville", "editaldeste", "execucao", "instituto",
    "forma", "meses", "referencia"
]])
stopwords_custom = list(set(stopwords_custom))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/rislamiranda/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Pipeline de treinamento de modelos

In [23]:
# Colunas
text_col = 'objeto_contrato'
num_cols = ['valorInicialCompra', 'valorFinalCompra', 'ano_assinatura', 'mes_assinatura', 'dias_ate_inicio_vigencia']
cat_cols = ['cd_orgao_siafi']
target = 'id_terc_count'

In [24]:
# Separar X e y
X = df_contrato_agg[[text_col] + num_cols + cat_cols].copy()
y = df_contrato_agg[target].copy()

In [25]:
# Pré-processamento com TF-IDF, numéricos e categóricos
preprocessor = ColumnTransformer([
    ('tfidf', TfidfVectorizer(max_features=50, stop_words=stopwords_custom), text_col),
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
], remainder='drop')

In [26]:
# Pré-processamento
preprocessor = ColumnTransformer([
    ('tfidf', TfidfVectorizer(stop_words=stopwords_custom), text_col),
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
], sparse_threshold=0)

In [27]:
# Métricas personalizadas
scoring = {
    'r2': make_scorer(r2_score),
    'rmse': make_scorer(root_mean_squared_error)
}

In [28]:
# Modelos com hiperparâmetros
modelos = {
    'Random Forest': {
        'model': RandomForestRegressor(random_state=42),
        'params': {'regressor__n_estimators': [50, 100], 'regressor__max_depth': [None, 10]}
    },
    'Gradient Boosting': {
        'model': GradientBoostingRegressor(random_state=42),
        'params': {'regressor__n_estimators': [50, 100], 'regressor__learning_rate': [0.05, 0.1]}
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {'regressor__n_neighbors': [3, 5, 7]}
    },
    'XGBoost': {
        'model': XGBRegressor(random_state=42),
        'params': {'regressor__n_estimators': [50, 100], 'regressor__max_depth': [3, 5]}
    },
    'Linear Regression': {
        'model': LinearRegression(),
        'params': {}  # sem hiperparâmetros
    }
}

In [29]:
# Treinamento com cross validation + GridSearch
resultados = {}
for nome, config in modelos.items():
    pipeline = Pipeline([
        ('preprocessamento', preprocessor),
        ('regressor', config['model'])
    ])

    print(f"\n🔍 Buscando hiperparâmetros para {nome}...")
    grid = GridSearchCV(
        pipeline,
        param_grid=config['params'],
        scoring='r2',
        cv=5,
        refit=True,
        n_jobs=-1
    )
    grid.fit(X, y)
    y_pred = grid.predict(X)
    resultados[nome] = {
        'melhor_score_cv': grid.best_score_,
        'melhores_parametros': grid.best_params_,
        'r2_score': r2_score(y, y_pred),
        'rmse': root_mean_squared_error(y, y_pred),
        'pipeline': grid.best_estimator_
    }
    print(f"✅ Melhor R² CV: {grid.best_score_:.4f} | RMSE: {resultados[nome]['rmse']:.2f}")

    # Importância das variáveis
    reg = grid.best_estimator_.named_steps['regressor']
    if hasattr(reg, 'feature_importances_'):
        feature_names = (
            grid.best_estimator_.named_steps['preprocessamento'].named_transformers_['tfidf'].get_feature_names_out().tolist()
            + num_cols
            + grid.best_estimator_.named_steps['preprocessamento'].named_transformers_['cat'].get_feature_names_out(cat_cols).tolist()
        )
        imp_df = pd.DataFrame({
            'Variavel': feature_names,
            'Importancia': reg.feature_importances_
        }).sort_values(by='Importancia', ascending=False).head(10)
        print("\n⭐️ Top 10 variáveis:")
        print(imp_df)


🔍 Buscando hiperparâmetros para Random Forest...
✅ Melhor R² CV: -0.0184 | RMSE: 347.18

⭐️ Top 10 variáveis:
                 Variavel  Importancia
405    valorInicialCompra     0.418025
423  cd_orgao_siafi_26260     0.066789
406      valorFinalCompra     0.047947
11                alfenas     0.041626
383                unifal     0.039258
305               projeto     0.028094
105          dependências     0.027888
311            realização     0.027293
232             materiais     0.027089
84             contratada     0.021465

🔍 Buscando hiperparâmetros para Gradient Boosting...
✅ Melhor R² CV: 0.0170 | RMSE: 347.92

⭐️ Top 10 variáveis:
                     Variavel  Importancia
405        valorInicialCompra     0.571131
118                  diversos     0.114629
123                      doze     0.047119
409  dias_ate_inicio_vigencia     0.046664
323                  regional     0.031523
19                      apoio     0.025616
406          valorFinalCompra     0.023967
12